<a href="https://colab.research.google.com/github/adarsh-meher/MachineHack---Predict-News-Category/blob/master/MH_Predict_News_category.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#!pip install fasttext
!pip install torchtext

In [2]:
!git clone https://github.com/adarsh-meher/MachineHack---Predict-News-Category.git
import os
os.chdir('/content/MachineHack---Predict-News-Category/')

Cloning into 'MachineHack---Predict-News-Category'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 15 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), done.


In [3]:
import torch
import pandas as pd
import numpy as np
import scipy as sc
import seaborn as sb
import matplotlib.pyplot as plt
import time
import nltk
import gensim as G
import spacy as S
import fasttext as ft

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
from sklearn.model_selection import train_test_split

In [0]:
news_data = pd.ExcelFile('Data_Train.xlsx').parse('Sheet1')

In [6]:
news_data.head()

,STORY,SECTION
0,But the most painful was the huge reversal in ...,3
1,How formidable is the opposition alliance amon...,0
2,Most Asian currencies were trading lower today...,3
3,"If you want to answer any question, click on ‘...",1
4,"In global markets, gold prices edged up today ...",3


In [16]:
#### Counts of news category
print(news_data['SECTION'].value_counts()/news_data.shape[0])

'''
Politics: 0
Technology: 1
Entertainment: 2
Business: 3
'''

### no imbalance in categories

target_to_name_map = {0:'Politics',1:"Technology",2:"Entertainment",3:"Business"}

1    0.363398
2    0.252229
0    0.221028
3    0.163346
Name: SECTION, dtype: float64


In [0]:
#### Creating a test set assuming similar proportion of categories in target
news_train,news_test,target_train,target_test = train_test_split(news_data.drop(['SECTION'],axis = 1),news_data['SECTION'],test_size = 0.2,stratify = news_data['SECTION'])
news_train.index,target_train.index = range(news_train.shape[0]),range(news_train.shape[0])
news_test.index,target_test.index = range(news_test.shape[0]),range(news_test.shape[0])

## ***Analyzing news_train texts***

In [0]:
df = pd.concat([news_train.rename(columns = {'STORY' : 'text'}),pd.Series(target_train,name = 'target')],axis = 1)

In [10]:
df.head()

,text,target
0,During the first round of the Amazon Great Ind...,1
1,The Trump White House has said in the past tha...,1
2,"“For example, I will be watching a movie and I...",1
3,It’s mostly driven by ultra high net worth ind...,1
4,I am at a stage right now where I am ready to ...,2


In [11]:
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,LancasterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download("stopwords")
nltk.download("punkt")
stop_words = set(stopwords.words("english"))
ps = PorterStemmer()
ls = LancasterStemmer()
wns = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
##### Freq Dist of all words for all categories except stopwords
tokens = [ y for x in df['text'].values for y in word_tokenize(x) if y.lower() not in stop_words and y.isalpha()]
tokens_fdist = FreqDist(tokens)

In [17]:
#### Freq distribution of words for each category
token_cat = {}
token_cat_fdist = {}
for i in df['target'].unique():
  print(target_to_name_map[i])
  token_cat[target_to_name_map[i]] = [ y for x in df[df['target'] == i]['text'].values for y in word_tokenize(x) if y.lower() not in stop_words and y.isalpha() ]
  token_cat_fdist[target_to_name_map[i]] = FreqDist(token_cat[target_to_name_map[i]]) 

Technology
Entertainment
Business
Politics


In [0]:
#### Dataframe to show for each category the top n words
n = 100
top_words_each_cat = pd.DataFrame({"Technology" : [i for i,v in token_cat_fdist["Technology"].most_common(n)],
                                      "Politics" : [i for i,v in token_cat_fdist["Politics"].most_common(n)],
                                      "Entertainment" : [i for i,v in token_cat_fdist["Entertainment"].most_common(n)],
                                      "Business" : [i for i,v in token_cat_fdist["Business"].most_common(n)]},index=range(n))

In [19]:
top_words_each_cat

,Technology,Politics,Entertainment,Business
0,said,BJP,film,said
1,also,Congress,said,year
2,Apple,said,also,market
3,India,party,like,India
4,new,Modi,actor,crore
...,...,...,...,...
95,design,manifesto,stars,Sebi
96,years,public,Also,first
97,top,many,always,investment
98,many,scheme,directed,supply


In [0]:
#### Extract set of words that appear both of any of the two categories
from itertools import combinations
common_words = []
for i in combinations(target_to_name_map.values(),2):
  [ common_words.append(x) for x in set(top_words_each_cat[i[0]]).intersection(set(top_words_each_cat[i[1]])) ]

common_words = set(list(common_words))

In [0]:
#### Dataframe to show for each category the bottom n words
n = 100
least_words_each_cat = pd.DataFrame({"Technology" : [i for i,v in token_cat_fdist["Technology"].most_common()[-n:]],
                                      "Politics" : [i for i,v in token_cat_fdist["Politics"].most_common()[-n:]],
                                      "Entertainment" : [i for i,v in token_cat_fdist["Entertainment"].most_common()[-n:]],
                                      "Business" : [i for i,v in token_cat_fdist["Business"].most_common()[-n:]]},index=range(n))

In [22]:
least_words_each_cat

,Technology,Politics,Entertainment,Business
0,Tunable,dalals,gaining,Concurrently
1,Xiang,devising,bankroll,arrested
2,Letting,Thirdly,Class,Solosec
3,Citrix,dilemmas,Sabharwal,Anshuman
4,XenMobile,Internal,Shoes,Investec
...,...,...,...,...
95,fuzzier,Buddha,Kamasutra,weed
96,trendsetters,Vidharbha,cardiac,killer
97,goggle,Bhagwat,Indiatoday,cancer
98,Ambeo,champion,sorry,opposition


### **`Classification using Torch`**

In [0]:
import torch
import torch.nn as nn
import  torchtext as ttext
from torchtext.data import Field,BucketIterator,TabularDataset

In [0]:
##### Create field object to specify how to process text
text_proc = Field(tokenize=word_tokenize)
label_proc = ttext.data.LabelField(dtype = torch.float32)

In [0]:
datafields = [('STORY',text_proc),('SECTION',label_proc)]

In [0]:
news_data_train = pd.concat([news_train,target_train],axis = 1)
news_data_test = pd.concat([news_test,target_test],axis = 1)
news_data_train.to_csv('News_Train.csv',index = False)
news_data_test.to_csv('News_Test.csv',index = False)

In [0]:
train,test = ttext.data.TabularDataset.splits(path = '/content/MachineHack---Predict-News-Category/',format = "csv",train = 'News_Train.csv',test = 'News_Test.csv',
                                       fields = datafields)

In [0]:
text_proc.build_vocab(train,max_size = 5000)
label_proc.build_vocab(train)

In [0]:
batch_size = 64
train_iter,test_iter = ttext.data.BucketIterator.splits((train,test),batch_size = batch_size,
                                                            sort_key = lambda x: len(x.STORY),
                                                            sort_within_batch = False)

In [0]:
class RNN(nn.Module):
  def __init__(self,input_dim,embed_dim,hidden_dim,output_dim):
    super(RNN,self).__init__()

    self.embed = nn.Embedding(input_dim,embed_dim)
    self.hidden = nn.RNN(embed_dim,hidden_dim)
    self.output = nn.Linear(hidden_dim,output_dim)

  def forward(self,text):
    embedding = self.embed(text)
    output,hidden = self.rnn(embedding)
    hidden_flat = hidden.squeeze(0)
    return self.output(hidden_flat)

In [0]:
input_dim = len(text_proc.vocab)
embed_dim = 200
hidden_dim = 300
output_dim = len(label_proc.vocab)

In [0]:
model = RNN(input_dim,embed_dim,hidden_dim,output_dim)

In [0]:
from torch.optim import Adam
optimizer = Adam(model.parameters(),lr = 0.0001)
criterion = nn.CrossEntropyLoss()